In [0]:
storageAccountName = ""
storageAccountAccessKey = ""
sasToken = ""
def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

In [0]:
#dbutils.fs.unmount('/mnt/datalakec47b4098c7f22fbc/landing-zone')
#dbutils.fs.unmount('/mnt/datalakec47b4098c7f22fbc/bronze')
#dbutils.fs.unmount('/mnt/datalakec47b4098c7f22fbc/silver')
#dbutils.fs.unmount('/mnt/datalakec47b4098c7f22fbc/gold')
'''
containers = ["landing-zone", "bronze", "silver", "gold"]

for container in containers:
    try:
        dbutils.fs.unmount(f"/mnt/{storageAccountName}/{container}")
        print(f"{container} desmontado com sucesso.")
    except Exception as e:
        print(f"Falha ao desmontar {container}:", e)
'''

In [0]:
mount_adls('landing-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

In [0]:
# Vendedores
df_vendedores_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/vendedores.csv")
colunas_vendedores = ["id", "nome", "email", "telefone", "data_cadastro"]
df_vendedores = df_vendedores_raw.toDF(*colunas_vendedores)

# Clientes
df_clientes_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/clientes.csv")
colunas_clientes = ["id", "nome", "email", "telefone", "data_cadastro"]
df_clientes = df_clientes_raw.toDF(*colunas_clientes)

# Categorias
df_categorias_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/categorias.csv")
colunas_categorias = ["id", "nome", "descricao"]
df_categorias = df_categorias_raw.toDF(*colunas_categorias)

# Estoque
df_estoque_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/estoque.csv")
colunas_estoque = ["id", "produto_id", "quantidade"]
df_estoque = df_estoque_raw.toDF(*colunas_estoque)

# Pagamentos
df_pagamentos_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/pagamentos.csv")
colunas_pagamentos = ["id", "pedido_id", "forma_pagamento_id", "valor_pago", "data_pagamento", "status"]
df_pagamentos = df_pagamentos_raw.toDF(*colunas_pagamentos)

# Entregas
df_entregas_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/entregas.csv")
colunas_entregas = ["id", "pedido_id", "transportadora_id", "data_envio", "data_entrega", "status"]
df_entregas = df_entregas_raw.toDF(*colunas_entregas)

# Avaliacoes
df_avaliacoes_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/avaliacoes.csv")
colunas_avaliacoes = ["id", "cliente_id", "produto_id", "nota", "comentario", "data_avaliacao"]
df_avaliacoes = df_avaliacoes_raw.toDF(*colunas_avaliacoes)

# Pedidos
df_pedidos_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/pedidos.csv")
colunas_pedidos = ["id", "cliente_id", "endereco_entrega_id", "data_pedido", "status", "total"]
df_pedidos = df_pedidos_raw.toDF(*colunas_pedidos)

# Enderecos_cliente
df_enderecos_cliente_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/enderecos_cliente.csv")
colunas_enderecos_cliente = ["id", "cliente_id", "logradouro", "numero", "complemento", "bairro", "cidade", "estado", "cep"]
df_enderecos_cliente = df_enderecos_cliente_raw.toDF(*colunas_enderecos_cliente)

# Transportadoras
df_transportadoras_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/transportadoras.csv")
colunas_transportadoras = ["id", "nome", "telefone", "email"]
df_transportadoras = df_transportadoras_raw.toDF(*colunas_transportadoras)

# Formas_pagamento
df_formas_pagamento_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/formas_pagamento.csv")
colunas_formas_pagamento = ["id", "descricao"]
df_formas_pagamento = df_formas_pagamento_raw.toDF(*colunas_formas_pagamento)

# Itens_pedido
df_itens_pedido_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/itens_pedido.csv")
colunas_itens_pedido = ["id", "pedido_id", "produto_id", "quantidade", "preco_unitario"]
df_itens_pedido = df_itens_pedido_raw.toDF(*colunas_itens_pedido)

# Produtos
df_produtos_raw = spark.read.option("header", "false").csv(f"/mnt/{storageAccountName}/landing-zone/ecommerce/produtos.csv")
colunas_produtos = ["id", "vendedor_id", "categoria_id", "nome", "descricao", "preco", "data_cadastro"]
df_produtos = df_produtos_raw.toDF(*colunas_produtos)


In [0]:
from pyspark.sql.functions import current_timestamp, lit

df_vendedores        = df_vendedores.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("vendedores.csv"))
df_clientes          = df_clientes.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("clientes.csv"))
df_categorias        = df_categorias.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("categorias.csv"))
df_estoque           = df_estoque.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("estoque.csv"))
df_pagamentos        = df_pagamentos.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("pagamentos.csv"))
df_entregas          = df_entregas.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("entregas.csv"))
df_avaliacoes        = df_avaliacoes.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("avaliacoes.csv"))
df_formas_pagamento  = df_formas_pagamento.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("formas_pagamento.csv"))
df_transportadoras   = df_transportadoras.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("transportadoras.csv"))
df_pedidos           = df_pedidos.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("pedidos.csv"))
df_enderecos_cliente = df_enderecos_cliente.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("enderecos_cliente.csv"))
df_itens_pedido      = df_itens_pedido.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("itens_pedido.csv"))
df_produtos          = df_produtos.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("produtos.csv"))


In [0]:
def sanitize_columns(df):
    for col_name in df.columns:
        new_name = (
            col_name.strip()
            .lower()
            .replace(" ", "_")
            .replace("(", "")
            .replace(")", "")
            .replace("-", "_")
            .replace(",", "")
            .replace(";", "")
            .replace("{", "")
            .replace("}", "")
            .replace("=", "")
            .replace("\n", "")
            .replace("\t", "")
        )
        df = df.withColumnRenamed(col_name, new_name)
    return df
dfs = [
    (df_vendedores, "vendedores"),
    (df_clientes, "clientes"),
    (df_categorias, "categorias"),
    (df_estoque, "estoque"),
    (df_pagamentos, "pagamentos"),
    (df_entregas, "entregas"),
    (df_avaliacoes, "avaliacoes"),
    (df_formas_pagamento, "formas_pagamento"),
    (df_transportadoras, "transportadoras"),
    (df_pedidos, "pedidos"),
    (df_enderecos_cliente, "enderecos_cliente"),
    (df_itens_pedido, "itens_pedido"),
    (df_produtos, "produtos"),
]

for df, name in dfs:
    df_sanitized = sanitize_columns(df)
    path = f"/mnt/{storageAccountName}/bronze/ecommerce/{name}"
    df_sanitized.write.format("delta").save(path)
